# Current To-Do:
- Sanity checks on trial beta maps
    Within-subject decoding
    Contrast map averaging and comparison
- Conjunction analysis

Backburner questions:
- XCP-D?
- DiFuMo atlas instead of Schaefer?
- RT adjustment of trial activity modeling
- Different SRM distance penalties (distance as penalty instead of parcelwise? Searchlights instead of parcels?)

# Testing

In [ ]:
from task_decoding import *
task = 'goNogo'
correct_only = False
subjects, data, labels = load_files(task, correct_only=correct_only)
data_srm = srm_transform(data, subjects)
task_aucs_srm, task_cms_srm = loso_cv(data_srm, labels, subjects)
task_aucs_nosrm, task_cms_nosrm = loso_cv(data, labels, subjects)